In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('spreadspoke_scores.csv')
df.dtypes

schedule_date           object
schedule_season          int64
schedule_week           object
team_home               object
team_away               object
stadium                 object
team_favorite_id        object
spread_favorite        float64
over_under_line         object
weather_detail          object
weather_temperature    float64
weather_wind_mph       float64
weather_humidity        object
score_home               int64
score_away               int64
stadium_neutral           bool
schedule_playoff          bool
game_id                 object
dtype: object

In [3]:
#drop columns with missing values
df = df.dropna(subset=['team_favorite_id', 'spread_favorite', 'over_under_line'])
df.index= range(0,len(df))
df = df.drop(columns=['game_id','stadium_neutral','schedule_playoff', 'stadium','schedule_date'])

In [4]:
df['weather_humidity']=df['weather_humidity'].apply(pd.to_numeric, errors='coerce')
df['over_under_line']=df['over_under_line'].apply(pd.to_numeric, errors='coerce')
#df['schedule_date']=df['schedule_date'].apply(pd.to_datetime)

In [5]:
#cleaning team_fav_id to correctly match team names in instances where team moved city
df.loc[df.team_favorite_id =='SD','team_favorite_id'] = 'LAC'
df.loc[(df.team_favorite_id=='LAR') & (df.team_home!= 'Los Angeles Rams') & (df.team_away!= 'Los Angeles Rams') & (df.team_home!= 'St. Louis Rams') & (df.team_away!= 'St. Louis Rams'), 'team_favorite_id']='OAK'
df.loc[((df.team_home=='Baltimore Colts') | (df.team_away=='Baltimore Colts')) & (df.team_favorite_id=='BAL'), 'team_favorite_id'] = 'IND'

In [6]:
special_weather_df = df[(df.weather_detail.isna() == False) & (df.weather_detail != 'DOME')]

In [7]:
#convert playoff rounds into ints (started at 19 bc seasons '93 and '99 had 18 weeks)
df.loc[(df.schedule_week=='WildCard') | (df.schedule_week=='Wildcard'), 'schedule_week']=19
df.loc[(df.schedule_week=='Division'), 'schedule_week']=20
df.loc[(df.schedule_week=='Conference'), 'schedule_week']=21
df.loc[(df.schedule_week=='Superbowl') | (df.schedule_week=='SuperBowl'), 'schedule_week']=22

#convert into ints
df.schedule_week = df.schedule_week.astype('int64')

In [8]:
df['point_total'] = df['score_home'] + df['score_away']

In [9]:
df['over_under_result'] = np.where(df['point_total']>df['over_under_line'], 'over', 'under')
df['over_under_result'] = np.where(df['point_total']==df['over_under_line'], 'push', df['over_under_result'])

In [10]:
df['score_difference'] = df['score_home'] - df['score_away']

In [11]:
teams_dict = {'Baltimore Ravens': 'BAL', 'Jacksonville Jaguars': 'JAX',
 'Carolina Panthers': 'CAR', 'Green Bay Packers': 'GB','St. Louis Cardinals': 'ARI',
 'Los Angeles Raiders': 'OAK','Detroit Lions': 'DET','Cincinnati Bengals': 'CIN',
 'Los Angeles Rams': 'LAR','Oakland Raiders': 'OAK',
 'Houston Texans': 'HOU', 'Tennessee Titans': 'TEN', 'New York Jets': 'NYJ',
 'St. Louis Rams': 'LAR', 'Baltimore Colts': 'IND', 'Los Angeles Chargers': 'LAC',
 'Phoenix Cardinals': 'ARI', 'San Diego Chargers': 'LAC', 'Indianapolis Colts': 'IND',
 'Washington Redskins': 'WAS', 'Miami Dolphins': 'MIA', 'Buffalo Bills': 'BUF',
 'Tennessee Oilers': 'TEN', 'Minnesota Vikings': 'MIN', 'Atlanta Falcons': 'ATL',
 'New Orleans Saints': 'NO', 'Pittsburgh Steelers': 'PIT','Kansas City Chiefs': 'KC', 
 'Denver Broncos': 'DEN', 'New York Giants': 'NYG', 'Houston Oilers': 'TEN',
 'Cleveland Browns': 'CLE', 'Philadelphia Eagles': 'PHI', 'Dallas Cowboys': 'DAL',
 'Arizona Cardinals': 'ARI', 'Chicago Bears': 'CHI', 'New England Patriots': 'NE',
 'San Francisco 49ers': 'SF','Tampa Bay Buccaneers': 'TB','Seattle Seahawks': 'SEA'}
teams_dict

{'Baltimore Ravens': 'BAL',
 'Jacksonville Jaguars': 'JAX',
 'Carolina Panthers': 'CAR',
 'Green Bay Packers': 'GB',
 'St. Louis Cardinals': 'ARI',
 'Los Angeles Raiders': 'OAK',
 'Detroit Lions': 'DET',
 'Cincinnati Bengals': 'CIN',
 'Los Angeles Rams': 'LAR',
 'Oakland Raiders': 'OAK',
 'Houston Texans': 'HOU',
 'Tennessee Titans': 'TEN',
 'New York Jets': 'NYJ',
 'St. Louis Rams': 'LAR',
 'Baltimore Colts': 'IND',
 'Los Angeles Chargers': 'LAC',
 'Phoenix Cardinals': 'ARI',
 'San Diego Chargers': 'LAC',
 'Indianapolis Colts': 'IND',
 'Washington Redskins': 'WAS',
 'Miami Dolphins': 'MIA',
 'Buffalo Bills': 'BUF',
 'Tennessee Oilers': 'TEN',
 'Minnesota Vikings': 'MIN',
 'Atlanta Falcons': 'ATL',
 'New Orleans Saints': 'NO',
 'Pittsburgh Steelers': 'PIT',
 'Kansas City Chiefs': 'KC',
 'Denver Broncos': 'DEN',
 'New York Giants': 'NYG',
 'Houston Oilers': 'TEN',
 'Cleveland Browns': 'CLE',
 'Philadelphia Eagles': 'PHI',
 'Dallas Cowboys': 'DAL',
 'Arizona Cardinals': 'ARI',
 'Chicago 

In [12]:
#create home and away id columns
df['home_id'] = ''
df['away_id'] = ''
df['home_id'] = df.team_home.map(teams_dict)
df['away_id'] = df.team_away.map(teams_dict)


In [13]:
df['home_favorite'] = 0
df.loc[df.home_id == df.team_favorite_id,'home_favorite'] = 1
df

schedule_season  schedule_week             team_home  \
0                1967             22     Green Bay Packers   
1                1968             22       Baltimore Colts   
2                1969             22    Kansas City Chiefs   
3                1970             22       Baltimore Colts   
4                1971             22        Dallas Cowboys   
5                1972             22        Miami Dolphins   
6                1973             22        Miami Dolphins   
7                1974             22     Minnesota Vikings   
8                1975             22        Dallas Cowboys   
9                1976             22     Minnesota Vikings   
10               1977             22        Dallas Cowboys   
11               1978             22        Dallas Cowboys   
12               1979              1  Tampa Bay Buccaneers   
13               1979              1         Buffalo Bills   
14               1979              1         Chicago Bears   
15               1979              1        Denver Broncos   
16               1979              1    Kansas City Chiefs   
17               1979              1      Los Angeles Rams   
18               1979              1     Minnesota Vikings   
19               1979              1    New Orleans Saints   
20               1979              1         New York Jets   
21               1979              1   Philadelphia Eagles   
22               1979              1      Seattle Seahawks   
23               1979              1   St. Louis Cardinals   
24               1979              1   Washington Redskins   
25               1979              1  New England Patriots   
26               1979              2        Denver Broncos   
27               1979              2       Baltimore Colts   
28               1979              2         Buffalo Bills   
29               1979              2         Chicago Bears   
...               ...            ...                   ...   
9625             2017             16   Washington Redskins   
9626             2017             16        Houston Texans   
9627             2017             16   Philadelphia Eagles   
9628             2017             17       Atlanta Falcons   
9629             2017             17      Baltimore Ravens   
9630             2017             17        Denver Broncos   
9631             2017             17         Detroit Lions   
9632             2017             17    Indianapolis Colts   
9633             2017             17  Los Angeles Chargers   
9634             2017             17      Los Angeles Rams   
9635             2017             17        Miami Dolphins   
9636             2017             17     Minnesota Vikings   
9637             2017             17  New England Patriots   
9638             2017             17       New York Giants   
9639             2017             17   Philadelphia Eagles   
9640             2017             17   Pittsburgh Steelers   
9641             2017             17      Seattle Seahawks   
9642             2017             17  Tampa Bay Buccaneers   
9643             2017             17      Tennessee Titans   
9644             2017             19    Kansas City Chiefs   
9645             2017             19      Los Angeles Rams   
9646             2017             19  Jacksonville Jaguars   
9647             2017             19    New Orleans Saints   
9648             2017             20  New England Patriots   
9649             2017             20   Philadelphia Eagles   
9650             2017             20     Minnesota Vikings   
9651             2017             20   Pittsburgh Steelers   
9652             2017             21  New England Patriots   
9653             2017             21   Philadelphia Eagles   
9654             2017             22  New England Patriots   

                 team_away team_favorite_id  spread_favorite  over_under_line  \
0          Oakland Raiders               GB            -13.5             43.0   

In [14]:
#determine whether favorite covered the spread, or push if even
df['favorite_covered'] = np.where((df.home_favorite==1) & (np.abs(df.score_difference) > np.abs(df.spread_favorite)),1,0)
df['favorite_covered'] = np.where((np.abs(df.score_difference) == np.abs(df.spread_favorite)),'2',df.favorite_covered)


In [15]:
df.tail()

schedule_season  schedule_week             team_home  \
9650             2017             20     Minnesota Vikings   
9651             2017             20   Pittsburgh Steelers   
9652             2017             21  New England Patriots   
9653             2017             21   Philadelphia Eagles   
9654             2017             22  New England Patriots   

                 team_away team_favorite_id  spread_favorite  over_under_line  \
9650    New Orleans Saints              MIN             -5.5             46.5   
9651  Jacksonville Jaguars              PIT             -7.0             40.5   
9652  Jacksonville Jaguars               NE             -7.5             46.0   
9653     Minnesota Vikings              MIN             -3.0             39.0   
9654   Philadelphia Eagles               NE             -4.5             48.5   

     weather_detail  weather_temperature  weather_wind_mph  weather_humidity  \
9650           DOME                 72.0               0.0               NaN   
9651            NaN                 18.0               2.0               NaN   
9652            NaN                 48.0               2.0               NaN   
9653            NaN                 46.0               1.0               NaN   
9654           DOME                 72.0               0.0               NaN   

      score_home  score_away  point_total over_under_result  score_difference  \
9650          29          24           53              over                 5   
9651          42          45           87              over                -3   
9652          24          20           44             under                 4   
9653          38           7           45              over                31   
9654          33          41           74              over                -8   

     home_id away_id  home_favorite favorite_covered  
9650     MIN      NO              1                0  
9651     PIT     JAX              1                0  
9652      NE     JAX              1                0  
9653     PHI     MIN              0                0  
9654      NE     PHI              1                1

In [16]:
#which team won the game outright (no spread)
df['winning_team'] = ''
df['winning_team'] = np.where(df.score_home > df.score_away, df.home_id, df.away_id)
df['winning_team'] = np.where(df.score_home == df.score_away, 'tie', df.winning_team)

df['losing_team'] = ''
df['losing_team'] = np.where(df.score_home < df.score_away, df.home_id, df.away_id)
df['losing_team'] = np.where(df.score_home == df.score_away, 'tie', df.losing_team)

In [17]:
df['home_wins'] = 0
df['home_losses'] = 0
df['home_ties'] = 0
df['away_wins'] = 0
df['away_losses'] = 0
df['away_ties'] = 0

In [18]:
sum((df['schedule_season'] == 2017) & (df['schedule_week'] < 5) & (df['winning_team'] == 'NE'))

2

In [19]:
def get_record(df):
    home_wins = []
    home_losses = []
    home_ties = []
    away_wins = []
    away_losses = []
    away_ties = []
    for index, row in df.iterrows():
        week = row.schedule_week
        season = row.schedule_season
        home_id = row.home_id
        away_id = row.away_id
        
        hw = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['winning_team'] == home_id))
        home_wins.append(hw)
        hl = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['losing_team'] == home_id))
        home_losses.append(hl)
        ht = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['winning_team'] == 'tie') & (df['home_id'] == home_id))
        home_ties.append(ht)
        
        aw = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['winning_team'] == away_id))
        away_wins.append(aw) 
        al = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['losing_team'] == away_id))
        away_losses.append(al)
        at = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['winning_team'] == 'tie') & (df['away_id'] == away_id))
        away_ties.append(at)
        
    return home_wins, home_losses, home_ties, away_wins, away_losses, away_ties
        
home_wins, home_losses, home_ties, away_wins, away_losses, away_ties = get_record(df)

In [20]:
df_filter = df.filter(items=['schedule_season', 'schedule_week', 'winning_team', 'home_id','home_wins'])

In [21]:
df['home_wins'] = home_wins
df['home_losses'] = home_losses
df['home_ties'] = home_ties
df['away_wins'] = away_wins
df['away_losses'] = away_losses
df['away_ties'] = away_ties

#df[(df.schedule_season==2017) & ((df.home_id == 'NYG') | (df.away_id == 'NYG'))]

In [22]:
def get_point_differential(df):
    result_array = np.empty((0,4))
    
    for index, row in df.iterrows():
        week = row.schedule_week
        season = row.schedule_season
        team_1 = row.home_id
        team_2 = row.away_id
        
        team_1_home_points=np.sum(df.score_home[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['home_id'] == team_1)])
        team_1_away_points=np.sum(df.score_away[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['away_id'] == team_1)])
        team_1_total_points = team_1_home_points+team_1_away_points
        
        team_1_home_points_against=np.sum(df.score_away[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['home_id'] == team_1)])
        team_1_away_points_against=np.sum(df.score_home[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['away_id'] == team_1)])
        team_1_total_points_against = team_1_home_points_against+team_1_away_points_against
        
        team_2_home_points=np.sum(df.score_home[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['home_id'] == team_2)])
        team_2_away_points=np.sum(df.score_away[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['away_id'] == team_2)])
        team_2_total_points = team_2_home_points+team_2_away_points
        
        team_2_home_points_against=np.sum(df.score_away[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['home_id'] == team_2)])
        team_2_away_points_against=np.sum(df.score_home[(df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['away_id'] == team_2)])
        team_2_total_points_against = team_2_home_points_against+team_2_away_points_against
    
        result_array = np.append(result_array, values=[[team_1_total_points, team_1_total_points_against, team_2_total_points, team_2_total_points_against]],axis=0)
        
    return result_array
        

In [23]:
point_diff_arr = get_point_differential(df)

In [24]:
point_diff_df = pd.DataFrame(point_diff_arr, columns=['home_points_for','home_points_against', 'away_points_for', 'away_points_against'], dtype='int')

In [25]:
df = pd.concat([df,point_diff_df],axis=1)

In [26]:

df['h_games'] = df['home_wins'] + df['home_losses'] + df['home_ties']
df['a_games'] = df['away_wins'] + df['away_losses'] + df['away_ties']

df['home_win_pct'] = np.round((df.home_wins+0.5*df.home_ties)/(df.h_games),3)
df['away_win_pct'] = np.round((df.away_wins+0.5*df.away_ties)/(df.a_games),3)
df['win_pct_diff'] = df.home_win_pct - df.away_win_pct

df['h_ppg'] = np.round((df.home_points_for/df.h_games),2)
df['h_papg'] = np.round((df.home_points_against/df.h_games),2)
df['a_ppg'] = np.round((df.away_points_for/df.a_games),2)
df['a_papg'] = np.round((df.away_points_against/df.a_games),2)

df['home_pt_diff_pg'] = np.round((df.home_points_for - df.home_points_against)/df.h_games, 3)
df['away_pt_diff_pg'] = np.round((df.away_points_for - df.away_points_against)/df.a_games, 3)
df['pt_diff_pg'] = df.home_pt_diff_pg - df.away_pt_diff_pg


In [27]:
def p_win_exp(df):
    array = np.empty((0,2))
    for index, row in df.iterrows():
        team1_pf = row.home_points_for
        team1_pa = row.home_points_against
        team2_pf = row.away_points_for
        team2_pa = row.away_points_against
        team_1_games = row.h_games + .001
        team_2_games = row.a_games + .001
        team1_exp = 1.5 * np.log((team1_pf + team1_pa)/team_1_games)
        team2_exp = 1.5 * np.log((team2_pf + team2_pa)/team_2_games)
        team1_pyth = ((team1_pf**team1_exp)/(team1_pf**team1_exp + team1_pa**team1_exp))
        team2_pyth = ((team2_pf**team2_exp)/(team2_pf**team2_exp + team2_pa**team2_exp))
        array = np.append(array, values = [[team1_pyth, team2_pyth]], axis = 0)
    return array

In [28]:
test = p_win_exp(df)

/Users/SamLiebman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
/Users/SamLiebman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/Users/SamLiebman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  if sys.path[0] == '':
/Users/SamLiebman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/Users/SamLiebman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
/Users/SamLiebman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in dou

In [29]:
test_df = pd.DataFrame(test, columns = ['home_exp_win_pct','away_exp_win_pct']) 
df = pd.concat([df,test_df], axis = 1)

In [30]:
df.corr()
#pd.plotting.scatter_matrix(df);

schedule_season  schedule_week  spread_favorite  \
schedule_season             1.000000       0.043183        -0.021448   
schedule_week               0.043183       1.000000        -0.083273   
spread_favorite            -0.021448      -0.083273         1.000000   
over_under_line             0.302191       0.005682        -0.063236   
weather_temperature         0.077884      -0.476890         0.061776   
weather_wind_mph           -0.232976       0.032504        -0.027542   
weather_humidity           -0.084873       0.009771        -0.000512   
score_home                  0.054854       0.042753        -0.147955   
score_away                  0.080308      -0.018517         0.122776   
point_total                 0.096494       0.018116        -0.021381   
score_difference           -0.016141       0.043065        -0.189302   
home_favorite              -0.038070       0.031014        -0.220783   
home_wins                   0.021888       0.811263        -0.190321   
home_losses                 0.023196       0.699103         0.090888   
home_ties                  -0.022884       0.063250         0.010790   
away_wins                   0.021441       0.811611         0.077849   
away_losses                 0.024212       0.737760        -0.221827   
away_ties                  -0.017683       0.065176         0.006601   
home_points_for             0.072680       0.917949        -0.153818   
home_points_against         0.078319       0.909777        -0.020823   
away_points_for             0.073080       0.919631         0.003099   
away_points_against         0.077644       0.918193        -0.133290   
h_games                     0.028665       0.982424        -0.077444   
a_games                     0.028431       0.983352        -0.079777   
home_win_pct               -0.004221       0.111980        -0.199969   
away_win_pct                0.003768       0.036674         0.248042   
win_pct_diff               -0.005907       0.053996        -0.320541   
h_ppg                       0.114653       0.111792        -0.213410   
h_papg                      0.118947      -0.052394         0.125125   
a_ppg                       0.106430       0.029174         0.222042   
a_papg                      0.117424      -0.016497        -0.167942   
home_pt_diff_pg             0.002758       0.112768        -0.231692   
away_pt_diff_pg            -0.001849       0.031233         0.265335   
pt_diff_pg                  0.002993       0.058157        -0.357234   
home_exp_win_pct            0.001200       0.117660        -0.244483   
away_exp_win_pct            0.002905       0.053879         0.277969   

                     over_under_line  weather_temperature  weather_wind_mph  \
schedule_season             0.302191             0.077884         -0.232976   
schedule_week               0.005682            -0.476890          0.032504   
spread_favorite            -0.063236             0.061776         -0.027542   
over_under_line             1.000000             0.155061         -0.238280   
weather_temperature         0.155061             1.000000         -0.416003   
weather_wind_mph           -0.238280            -0.416003          1.000000   
weather_humidity           -0.066874            -0.022677          0.036637   
score_home                  0.199866            -0.013868         -0.055085   
score_away                  0.192984             0.071623         -0.112542   
point_total                 0.281463             0.040395         -0.119360   
score_difference            0.009685            -0.059089          0.038314   
home_favorite              -0.013010            -0.030487          0.055630   
home_wins                   0.075649            -0.407645          0.054158   
home_losses                -0.068929            -0.341440          0.000112   
home_ties                  -0.002991            -0.061295          0.019954   
away_wins                   0.071624            -0.372205          0.014064   
away_losses  

In [31]:
# corr_mask = df.drop(columns=['score_difference','point_total', 'schedule_season', 'schedule_week','score_home', 'score_away',])
# # corr_mask['h_games'] = corr_mask['home_wins'] + corr_mask['home_losses'] + corr_mask['home_ties']
# # corr_mask['a_games'] = corr_mask['away_wins'] + corr_mask['away_losses'] + corr_mask['away_ties']

# # corr_mask['home_win_pct'] = np.round((corr_mask.home_wins+0.5*corr_mask.home_ties)/(corr_mask.h_games),3)
# # corr_mask['away_win_pct'] = np.round((corr_mask.away_wins+0.5*corr_mask.away_ties)/(corr_mask.a_games),3)
# # corr_mask['win_pct_diff'] = corr_mask.home_win_pct - corr_mask.away_win_pct

# # corr_mask['h_ppg'] = np.round((corr_mask.home_points_for/corr_mask.h_games),2)
# # corr_mask['h_papg'] = np.round((corr_mask.home_points_against/corr_mask.h_games),2)
# # corr_mask['a_ppg'] = np.round((corr_mask.away_points_for/corr_mask.a_games),2)
# # corr_mask['a_papg'] = np.round((corr_mask.away_points_against/corr_mask.a_games),2)

# # corr_mask['home_pt_diff_pg'] = np.round((corr_mask.home_points_for - corr_mask.home_points_against)/corr_mask.h_games, 3)
# # corr_mask['away_pt_diff_pg'] = np.round((corr_mask.away_points_for - corr_mask.away_points_against)/corr_mask.a_games, 3)
# # corr_mask['pt_diff_pg'] = corr_mask.home_pt_diff_pg - corr_mask.away_pt_diff_pg

# corr_mask.columns

# # df2 = corr_mask.drop(columns=['home_wins', 'home_losses', 'home_ties', 'away_wins', 
# #     'away_losses', 'away_ties', 'home_points_for', 'home_points_against','away_points_for', 
# #     'away_points_against', 'h_games', 'a_games', 'home_win_pct', 'away_win_pct',
# #     'home_pt_diff_pg','away_pt_diff_pg'])

In [32]:
# corr_mask.corr()

In [33]:
# df2.corr()
# pd.plotting.scatter_matrix(df2,figsize=(20,20),grid=True,diagonal='kde');

In [1]:
def get_record_last_4(df):
    result_array = np.empty((0,2))

    for index, row in df.iterrows():
        week = row.schedule_week
        season = row.schedule_season
        home_id = row.home_id
        away_id = row.away_id

        h_w_last_4 = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['schedule_week'] > (week-5)) & (df['winning_team'] == home_id))
        h_l_last_4 = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['schedule_week'] > (week-5)) & (df['winning_team'] == away_id))
        h_t_last_4 = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['schedule_week'] > (week-5)) & (df['winning_team'] == 'tie') & (df['home_id'] == home_id))

        h_win_pct_last_4 = np.round(((h_w_last_4 + 0.5*h_t_last_4) / 4),3)

        a_w_last_4 = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['schedule_week'] > (week-5)) & (df['winning_team'] == away_id))
        a_l_last_4 = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['schedule_week'] > (week-5)) & (df['winning_team'] == home_id))
        a_t_last_4 = sum((df['schedule_season'] == season) & (df['schedule_week'] < week) & (df['schedule_week'] > (week-5)) & (df['winning_team'] == 'tie') & (df['away_id'] == away_id))

        a_win_pct_last_4 = np.round(((a_w_last_4 + 0.5*a_t_last_4) / 4),3)

        result_array = np.append(result_array, values = [[h_win_pct_last_4, a_win_pct_last_4]], axis = 0)

    return result_array

last_4_arr = get_record_last_4(df)

NameError: name 'df' is not defined

In [ ]:
last_4_df = pd.DataFrame(last_4_arr,columns=['home_win_pct_last_4', 'away_win_pct_last_4'])
last_4_df

df = pd.concat([df,last_4_df],axis=1)

### To-do:
* convert points for and against, and points differential to per game. helps normalize data
* should we use diff or both home an away for each stat
* ppg and papg as indicators for o/u

In [34]:
df.to_csv('nfl_betting_df.csv')